In [18]:
import cv2
import numpy as np
large = cv2.imread('./image/namecard.png')
rgb = cv2.pyrDown(large)    #guassian pyramid image 크기 축속
small = cv2.cvtColor(rgb, cv2.COLOR_BGR2GRAY)   # 일단 gray로 변경

kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
grad = cv2.morphologyEx(small, cv2.MORPH_GRADIENT, kernel)  # erosion, dilation 적용
# cv2.MORPH_GRADIENT : 그레디언트로 엣지 추출
_, bw = cv2.threshold(grad, 0.0, 255.0, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9, 1))
connected = cv2.morphologyEx(bw, cv2.MORPH_CLOSE, kernel)

cv2.imshow('bw', bw)
cv2.imshow('connected', connected)
cv2.waitKey()
cv2.destroyAllWindows()

In [1]:
# using RETR_EXTERNAL instead of RETR_CCOMP
contours, hierarchy = cv2.findContours(connected.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
mask = np.zeros(bw.shape, dtype=np.uint8)
for idx in range(len(contours)):
    x, y, w, h = cv2.boundingRect(contours[idx])
    mask[y:y+h, x:x+w] = 0
    cv2.drawContours(mask, contours, idx, (255, 255, 255), -1)
    r = float(cv2.countNonZero(mask[y:y+h, x:x+w])) / (w * h)
    if r > 0.45 and w > 8 and h > 8:
        cv2.rectangle(rgb, (x, y), (x+w-1, y+h-1), (0, 255, 0), 2)
# show image with contours rect
cv2.imshow('rects', rgb)
cv2.waitKey()
cv2.destroyAllWindows()